In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.model_selection import train_test_split
import math
import unicodedata
import re

In [16]:
raw_data=pd.read_csv('../dataset/job_data_new.csv')

In [17]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26361 entries, 0 to 26360
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Job Name              26361 non-null  object
 1   Job Link              26361 non-null  object
 2   Location              26361 non-null  object
 3   Update Date           26361 non-null  object
 4   Deadline              26361 non-null  object
 5   Experience            26361 non-null  object
 6   Job Level             26361 non-null  object
 7   Industry              26361 non-null  object
 8   Employment Type       26359 non-null  object
 9   Welfare               26361 non-null  object
 10  Job Description       26361 non-null  object
 11  Job Requirements      26361 non-null  object
 12  Other Information     26361 non-null  object
 13  Job Tags              26361 non-null  object
 14  Company URL           26361 non-null  object
 15  Salary                26361 non-null

In [18]:
used_data=raw_data[["Location","Experience","Job Level","Employment Type" ,"Industry","Welfare","Job Description","Job Requirements","Other Information","Follower","Salary"]]

In [19]:
used_data.head()

,Location,Experience,Job Level,Employment Type,Industry,Welfare,Job Description,Job Requirements,Other Information,Follower,Salary
0,Hồ Chí Minh,3 - 5 \n ...,Quản lý,Nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Đồng phục, Chế độ th...",Tuyển dụng và đào tạo: Quản lý quy trình tuyển...,Trình độ: Tốt nghiệp cao đẳng/đại học chuyên n...,Bằng cấp:\n Đại học\nThời gian ...,NaN,Cạnh tranh
1,Hồ Chí Minh,Not specified,Nhân viên,Nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đồng phụ...","Thương lượng với khách hàng về giá cả, chất lư...","Tốt nghiệp đại học các chuyên ngành Kinh tế, n...",Bằng cấp:\n Đại học\nĐộ tuổi:\n...,NaN,Cạnh tranh
2,Not specified,Not specified,Not specified,Not specified,Not Found,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đào tạo,...",Not specified,Not specified,Bằng cấp:\n Trung cấp\nHình thứ...,NaN,Not specified
3,Bình Định,2 - 5 \n ...,Trưởng nhóm / Giám sát,Nhân viên chính thức,\n ...,"Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Xe ...",Job Description:\n1、Develop and optimize depar...,Experience Requirement：\n- Over 3 years...,Bằng cấp:\n Đại học\nĐộ tuổi:\n...,169,Cạnh tranh
4,Hà Nội,Trên 1 \n ...,Nhân viên,Nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",- Phát triển khách hàng; tìm kiếm khách hàng ...,- Tốt nghiệp Đại Học hệ chính qui thuộc các n...,Độ tuổi:\n Không giới hạn tuổi\...,2.27,Cạnh tranh


### Location

In [20]:
location_data=used_data.copy()


In [21]:
def remove_diacritics(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

In [22]:
location_data['Location'] = location_data['Location'].apply(lambda x: x.title())

location_data['Location'] = location_data['Location'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x.strip())
location_data['Location'] = location_data['Location'].apply(remove_diacritics)

In [23]:
city_to_number = {
    "Hai Duong": 0,
    "Long An": 1,
    "Ha Noi": 2,
    "Ho Chi Minh": 3,
    "Binh Duong": 4,
    "Đong Nai": 5,
    "Quang Ngai": 6,
    "Vinh Long": 7,
    "Lam Đong": 8,
    "Can Tho": 9,
    "Bac Ninh": 10,
    "Dak Lak": 11,
    "Gia Lai": 12,
    "Bac Giang": 13,
    "Binh Thuan": 14,
    "Tay Ninh": 15,
    "Ba Ria - Vung Tau": 16,
    "Ha Nam": 17,
    "Thai Binh": 18,
    "Bac Lieu": 19,
    "Thanh Hoa": 20,
    "Binh Đinh": 21,
    "Tra Vinh": 22,
    "Hoa Binh": 23,
    "Đa Nang": 24,
    "Hung Yen": 25,
    "Hai Phong": 26,
    "Kien Giang": 27,
    "Vinh Phuc": 28,
    "Ha Giang": 29,
    "Tuyen Quang": 30,
    "Đien Bien": 31,
    "Nam Dinh": 32,
    "Lang Son": 33,
    "Nghe An": 34,
    "Quang Tri": 35,
    "Quang Ninh": 36,
    "Ninh Thuan": 37,
    "Đong Thap": 38,
    "Quang Nam": 39,
    "Thua Thien- Hue": 40,
    "Tien Giang": 41,
    "Khanh Hoa": 42,
    "Phu Yen": 43,
    "Quang Binh": 44,
    "Hau Giang": 45,
    "Binh Phuoc": 46,
    "Phu Tho": 47,
    "Ninh Binh": 48,
    "Thai Nguyen": 49,
    "Bac Can": 50,
    "Son La": 51,
    "Ha Tinh": 52,
    "Yen Bai": 53,
    "Ca Mau": 54,
    "An Giang": 55,
    "Ben Tre": 56,
    "Lao Cai": 57,
    "Soc Trang": 58,
    "Cao Bang": 59,
    "Dak Nong": 60,
    "Kon Tum": 61,
    "Lai Chau": 62,
    "Nam Đinh":63,
    
    # Khu vực khác
    "Kv Đong Nam Bo": 64,
    "Kv Tay Nguyen": 65,
    "Kv Nam Trung Bo": 66,
    "Kv Bac Trung Bo": 67,
    "Đong Bang Song Cuu Long": 68,
    "Toan Quoc": 69,
    "Khac": 70,
    "Not Specified": 70,

    # Các địa điểm khác (được gộp chung vào "foreign city/province")
    "Svay Rieng": 71,
    "Tokyo": 72,
    "Xiangkhouang": 73,
    "Champasak": 74,
    "Yokohama": 75,
    "Hokkaido": 76,
    "Malaysia": 77,
    "Attapeu":78,
    "Bangkok": 79,
    "Kratie": 80,
    "Phnompenh": 81,
    "Vientiane":82,
    "Kuala Lumpur": 83,
    "Quoc Te": 84
}

In [24]:
location_data['Location'] = location_data['Location'].replace(city_to_number)

In [25]:
location=location_data['Location'].unique()
location

array([ 3, 70, 21,  2,  0, 55,  1, 36, 24,  4,  6,  5, 25,  7, 17,  8,  9,
       10, 11, 12, 42, 13, 14, 15, 16, 41, 18, 19, 20, 22, 37, 23, 46, 26,
       27, 59, 53, 45, 28, 29, 30, 31, 63, 33, 64, 34, 35, 78, 38, 39, 40,
       69, 43, 44, 82, 47, 56, 71, 48, 49, 50, 51, 52, 60, 54, 65, 57, 58,
       68, 84, 61, 72, 73, 79, 74, 75, 76, 77, 67, 80, 66, 62, 81, 83],
      dtype=int64)

### Experience

In [26]:
experience_data=location_data.copy()

In [27]:
def extract_number(value):
    if 'Chưa có kinh nghiệm' in value:
        return 0  # Nếu "Chưa có kinh nghiệm" trả về 0
    elif match := re.match(r'(\d+)\s*-\s*\d+', value):  # Nếu định dạng là number - number
        return int(match.group(1))  # Lấy số đầu tiên
    elif match := re.search(r'\d+', value):  # Nếu chỉ có một số duy nhất
        return int(match.group(0))  # Lấy số đó
    return 0

In [28]:
experience_data["Experience"]=experience_data["Experience"].str.replace("\n","",regex=True)
experience_data["Experience"]=experience_data["Experience"].str.replace("Năm","",regex=True)
experience_data["Experience"]=experience_data["Experience"].str.replace(r"\s+",' ',regex=True).str.strip()
experience_data["Experience"]=experience_data["Experience"].apply(extract_number)

In [29]:
experience=experience_data["Experience"].unique()
experience

array([ 3,  0,  2,  1,  5,  4,  7, 10,  8,  6, 15,  9, 12, 30, 20],
      dtype=int64)

### Job Level

In [30]:
job_level_data=experience_data.copy()



In [31]:
job_level_list = {
    'mới tốt nghiệp':1,
    'sinh viên/ thực tập sinh':1,
    'nhân viên':2,
    'trưởng nhóm':3,
    'trưởng nhóm / giám sát':3,
    'quản lý':4,
    'phó giám đốc':5,
    'giám đốc':6,
    'tổng giám đốc':7,
    'not specified':0
}

job_level_data['Job Level'] = job_level_data['Job Level'].str.lower()

job_level_data['Job Level'] = job_level_data['Job Level'].map(job_level_list)

In [32]:
job_level=job_level_data["Job Level"].unique()
job_level

array([4, 2, 0, 3, 6, 1, 5, 7], dtype=int64)

### Employment Type

In [33]:
employment_type_data=job_level_data.copy()

In [34]:
employment_type_data["Employment Type"]=employment_type_data["Employment Type"].str.lower()
employment_type_data["Employment Type"]=employment_type_data["Employment Type"].fillna("not specified")

employment=employment_type_data["Employment Type"].unique()
employment=pd.DataFrame(employment,columns=["Employment Type"])
employment = employment.dropna()
employment = employment[employment["Employment Type"] != "not specified"]

split_job_types = employment["Employment Type"].str.split(", ")

unique_job_types= set()
split_job_types.apply(unique_job_types.update)

unique_job_types=list(unique_job_types)

employment_type_data = employment_type_data.join(pd.DataFrame(columns=unique_job_types))

for job_type in unique_job_types:
    employment_type_data[job_type] = employment_type_data["Employment Type"].apply(
        lambda x: 1 if job_type in x else 0
    )

employment_type_data.rename(columns={'nhân viên chính thức': 'Employment Type: Official',
                                      "bán thời gian":"Employment Type: Part-time", 
                                      "thực tập": "Employment Type: Internship", 
                                      "thời vụ/ nghề tự do":"Employment Type: Freelance"}, inplace=True)

In [35]:
employment_type_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26361 entries, 0 to 26360
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Location                     26361 non-null  int64 
 1   Experience                   26361 non-null  int64 
 2   Job Level                    26361 non-null  int64 
 3   Employment Type              26361 non-null  object
 4   Industry                     26361 non-null  object
 5   Welfare                      26361 non-null  object
 6   Job Description              26361 non-null  object
 7   Job Requirements             26361 non-null  object
 8   Other Information            26361 non-null  object
 9   Follower                     25262 non-null  object
 10  Salary                       26361 non-null  object
 11  Employment Type: Internship  26361 non-null  int64 
 12  Employment Type: Part-time   26361 non-null  int64 
 13  Employment Type: Official    26

### Follower

In [36]:
follower_data=employment_type_data.copy()

In [37]:
follower_data["Follower"]=follower_data["Follower"].fillna("0")
follower_data["Follower"]=follower_data["Follower"].str.replace(".","",regex=True)
follower_data["Follower"]=follower_data["Follower"].str.replace("Not specified","0",regex=True)
follower_data["Follower"]=follower_data["Follower"].str.replace("Not Found","0",regex=True)
follower_data["Follower"]=follower_data["Follower"].astype(int)



In [38]:
follower_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26361 entries, 0 to 26360
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Location                     26361 non-null  int64 
 1   Experience                   26361 non-null  int64 
 2   Job Level                    26361 non-null  int64 
 3   Employment Type              26361 non-null  object
 4   Industry                     26361 non-null  object
 5   Welfare                      26361 non-null  object
 6   Job Description              26361 non-null  object
 7   Job Requirements             26361 non-null  object
 8   Other Information            26361 non-null  object
 9   Follower                     26361 non-null  int32 
 10  Salary                       26361 non-null  object
 11  Employment Type: Internship  26361 non-null  int64 
 12  Employment Type: Part-time   26361 non-null  int64 
 13  Employment Type: Official    26

In [39]:
follower_data.to_csv('../dataset/follower_data.csv',index=False)

### Salary

In [40]:
salary_data=follower_data.copy()   

In [41]:
# Hàm chuyển đổi chuỗi tiền tệ từ định dạng 'Tr' sang số và lấy giá trị nhỏ nhất
def convert_salary_to_min(salary):
    if(salary != 'cạnh tranh'):   
        if "lên đến"in salary and "usd" in salary:
            if "-" in salary:
                parts = salary.replace("lên đến", "").replace(" usd", "").replace(".","").split(" - ")
                min_salary = float(parts[0].replace(",", ""))*23000
                return min_salary
            else:
                value = salary.replace("trên", "").replace("lên đến", "").replace(" usd", "").replace(".","").strip()
                min_salary = float(value.replace(",", ""))*23000
                return min_salary
        
        elif "trên"in salary and "usd" in salary:
            value = salary.replace("trên", "").replace(" usd", "").replace(".","").strip()
            min_salary = float(value.replace(",", ""))*23000
            return min_salary
        
        elif 'usd' in salary and '-' in salary:
            parts = salary.replace(" usd", "").replace(" tr", "").replace(".","").split(" - ")
            min_salary = float(parts[0].replace(",", ""))*23000
            return min_salary
        
        elif 'usd' in salary:
            min_salary = float(salary.replace(" usd", "").replace(".","").replace(",", ""))*23000
            return min_salary
        
        elif 'trên' in salary and 'tr' in salary:
            if "-" in salary:
                parts = salary.replace("lên đến", "").replace("trên", "").replace(" tr", "").replace(" vnd","").split(" - ")
                min_salary = float(parts[0].replace(",", ".")) * 1_000_000
                return min_salary
            else:
                value = salary.replace("lên đến", "").replace("trên", "").replace(" tr", "").replace(" vnd","").strip()
                min_salary = float(value.replace(",", ".")) * 1_000_000
                return min_salary
        elif 'lên đến' in salary and "tr" in salary:
            if "-" in salary:
                parts = salary.replace("lên đến", "").replace(" tr", "").replace(" vnd","").split(" - ")
                min_salary = float(parts[0].replace(",", ".")) * 1_000_000
                return min_salary
            else:
                value = salary.replace("lên đến", "").replace(" tr", "").replace(" vnd","").strip()
                min_salary = float(value.replace(",", ".")) * 1_000_000
                return min_salary
        
        # Kiểm tra nếu là dạng 'Tr' VND
        elif 'tr' in salary and ' - ' in salary:
            # Loại bỏ " Tr VND", tách và nhân với 1,000,000
            parts = salary.replace(" tr", "").replace(" vnd","").split(" - ")
            min_salary = float(parts[0].replace(",", ".")) * 1_000_000
            return min_salary
        
        # Kiểm tra nếu là dạng 'Tr' VND
        elif 'tr' in salary:
            # Loại bỏ " Tr VND" và nhân với 1,000,000
            min_salary = float(salary.replace(" tr", "").replace(" vnd","")) * 1_000_000
            return min_salary
        
    return "Cạnh tranh"

In [42]:
salary_data['Salary'] = salary_data['Salary'].replace(
    ['0','Thỏa thuận', 'Negotiable', 'Not Specified', 'Not Found', 'not found', 'Not specifield'], 
    np.nan)
salary_data['Salary'] = salary_data['Salary'].str.lower()
salary_data['Salary'] = salary_data['Salary'].apply(convert_salary_to_min)

In [43]:
salary_data

,Location,Experience,Job Level,Employment Type,Industry,Welfare,Job Description,Job Requirements,Other Information,Follower,Salary,Employment Type: Internship,Employment Type: Part-time,Employment Type: Official,Employment Type: Freelance
0,3,3,4,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Đồng phục, Chế độ th...",Tuyển dụng và đào tạo: Quản lý quy trình tuyển...,Trình độ: Tốt nghiệp cao đẳng/đại học chuyên n...,Bằng cấp:\n Đại học\nThời gian ...,0,Cạnh tranh,0,0,1,0
1,3,0,2,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đồng phụ...","Thương lượng với khách hàng về giá cả, chất lư...","Tốt nghiệp đại học các chuyên ngành Kinh tế, n...",Bằng cấp:\n Đại học\nĐộ tuổi:\n...,0,Cạnh tranh,0,0,1,0
2,70,0,0,not specified,Not Found,"Chế độ bảo hiểm, Phụ cấp, Xe đưa đón, Đào tạo,...",Not specified,Not specified,Bằng cấp:\n Trung cấp\nHình thứ...,0,Cạnh tranh,0,0,0,0
3,21,2,3,nhân viên chính thức,\n ...,"Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Xe ...",Job Description:\n1、Develop and optimize depar...,Experience Requirement：\n- Over 3 years...,Bằng cấp:\n Đại học\nĐộ tuổi:\n...,169,Cạnh tranh,0,0,1,0
4,2,1,2,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",- Phát triển khách hàng; tìm kiếm khách hàng ...,- Tốt nghiệp Đại Học hệ chính qui thuộc các n...,Độ tuổi:\n Không giới hạn tuổi\...,227,Cạnh tranh,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26356,3,0,1,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Phụ cấp, Chế độ thưởng, Chăm ...","· Vận hành hệ thống bảo mật, giữ cho hệ ...",Điều kiện ứng tuyển\n· Là sinh viên tốt ...,Bằng cấp:\n Đại học\nĐộ tuổi:\n...,0,34500000.0,0,0,1,0
26357,19,0,1,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Chế độ thưở...","Với mục tiêu mở rộng thêm các Siêu thị, Team P...",- Nam từ 22 - 30 tuổi. Sức khỏe tốt\n- Không y...,Bằng cấp:\n Cao đẳng\nGiới tính...,20965,Cạnh tranh,0,0,1,0
26358,69,0,1,bán thời gian,\n ...,"Chế độ thưởng, Đào tạo, Tăng lương",- Dạy luyện thi chứng chỉ FE (chứng chỉ chuẩn ...,- Đã có chứng chỉ FE.\n- Ứng viên cần có kiến ...,Độ tuổi:\n Không giới hạn tuổi\...,98,7000000.0,0,1,0,0
26359,16,0,1,nhân viên chính thức,\n ...,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Chế độ thưở...","Với mục tiêu mở rộng thêm các Siêu thị, Team P...",- Nam từ 22 - 30 tuổi. Sức khỏe tốt\n- Không y...,Bằng cấp:\n Cao đẳng\nGiới tính...,20965,Cạnh tranh,0,0,1,0


In [47]:
salary_data.to_csv('../dataset/pre_translate_data.csv',index=False,encoding='utf-8-sig')    